# Read in the data

In [25]:
from eventcnn.datasources.davis import DavisDataset
from eventcnn.datasources.eventsource import EventSource
import tensorflow as tf
import numpy as np

In [26]:
dataset1 = DavisDataset.named_dataset("poster_translation")
dataset2 = DavisDataset.named_dataset("shapes_translation")

In [27]:
source1 = EventSource(dataset1, 
                      testing_fraction=0.2,
                      validation_fraction=0.4,
                      events_per_block=10000)
source2 = EventSource(dataset2, 
                      testing_fraction=0.2,
                      validation_fraction=0.4,
                      events_per_block=10000)

In [28]:
# The + operator combines event sources, respecting their 
# individual training/test/validation splits
combined = source1 + source2

Access the clusters like this:

In [29]:
elem = combined.training(shuffle=True).__next__()
sparse_tensor1 = elem.events_as_sparse_tensor_rescaled( n_layers = 5, scaling = 2)
print(sparse_tensor1.shape)
dense_tensor1 = tf.sparse_tensor_to_dense(sparse_tensor1)

[120, 90, 4]


In [31]:
elem.delta_position
combined.testing().__next__()

In [32]:
def get_data_batch(generator, batch_size):
    list_of_tensors = []
    list_of_truth = []
    num_blocks = 0
    for block in generator:
        num_blocks += 1
        if num_blocks > batch_size:
            break
        list_of_tensors.append(tf.sparse_tensor_to_dense(block.events_as_sparse_tensor_rescaled( n_layers = 5, scaling = 4)))
        list_of_truth.append(block.delta_position)
    X = tf.pack(list_of_tensors)
    y = tf.pack(list_of_truth)
    return X, y

In [34]:
X_test, y_test = get_data_batch(combined.testing(), 50)
# X_validation, y_validation = get_data_batch(combined.validation(), 5) TODO: Somehow does not work?!?!

In [13]:
print(X_test)

Tensor("pack_3:0", shape=(500, 60, 45, 4), dtype=float64)


In [19]:
num_blocks = 0
for block in combined.training(shuffle=True):
    num_blocks += 1
    if num_blocks > 10:
        break
    print(block.time_span)
    sparse_tensor = block.events_as_sparse_tensor_rescaled( n_layers = 5, scaling = 2)

[46.661661000000002, 46.666632999999997]
[38.087312001000001, 38.106126001]
[21.872201, 21.875403000999999]
[8.5099020000000003, 8.5233780010000011]
[59.509162001000007, 59.511696001000004]
[2.3797550009999999, 2.3878360010000002]
[0.36012300000000003, 0.37505699999999997]
[25.028206000000001, 25.032253999999998]
[59.289725001000001, 59.292600999999998]
[53.723846000000002, 53.727438001000003]


# Setup the network

In [45]:
# We'll bundle groups of examples during training for efficiency.
# This defines the size of the batch.
BATCH_SIZE = 60

# We aggregate our data to 5 time slices
N_TIME_SLICES = 5

# Inversely scaling the input image resolution
SCALING_FAC = 2

# Number of observed coordinates
NUM_LABELS = 3

# Resolution
X_RES = np.uint32(240 / SCALING_FAC)
Y_RES = np.uint32(180 / SCALING_FAC)

# The random seed that defines initialization.
SEED = 42

In [48]:
# This is where training samples and labels are fed to the graph.
# These placeholder nodes will be fed a batch of training data at each
# training step, which we'll write once we define the graph structure.
train_data_node = tf.placeholder(  tf.float32,
                                   shape=(BATCH_SIZE, Y_RES, X_RES, N_TIME_SLICES))
train_labels_node = tf.placeholder(tf.float32,
                                   shape=(BATCH_SIZE, NUM_LABELS))

# For the validation and test data, we'll just hold the entire dataset in
# one constant node.
# validation_data_node = tf.constant(combined.validation())
# test_data_node = tf.constant(X_test)

# The variables below hold all the trainable weights. For each, the
# parameter defines how the variables will be initialized.

# 5x5x5 filter, depth 32.
conv1_weights = tf.Variable(
  tf.truncated_normal([5, 5, 5, N_TIME_SLICES, 32],
                      stddev=0.1,
                      seed=SEED))
conv1_biases = tf.Variable(tf.zeros([32]))

conv2_weights = tf.Variable(
  tf.truncated_normal([5, 5, 5, 32, 64],
                      stddev=0.1,
                      seed=SEED))
conv2_biases = tf.Variable(tf.constant(0.1, shape=[64]))

# fully connected, depth 512.
fc1_weights = tf.Variable(  
  tf.truncated_normal([Y_RES // 4 * X_RES // 4 * 64, 512],
                      stddev=0.1,
                      seed=SEED))
fc1_biases = tf.Variable(tf.constant(0.1, shape=[512]))

# fully connected, depth 512.
fc2_weights = tf.Variable(
  tf.truncated_normal([512, NUM_LABELS],
                      stddev=0.1,
                      seed=SEED))
fc2_biases = tf.Variable(tf.constant(0.1, shape=[NUM_LABELS]))

print('Done')

Done


In [ ]:
def model(data, train=False):
    """The Model definition."""
    # 3D convolution, with 'SAME' padding (i.e. the output feature map has
    # the same size as the input). Note that {strides} is a 5D array whose
    # shape matches the data layout: [tensor_index, y, x, time_layer].
    conv = tf.nn.conv3d(data,
                        conv1_weights,
                        strides=[1, 1, 1, 1, 1],
                        padding='SAME')

    # Bias and rectified linear non-linearity.
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))

    # Max pooling. The kernel size spec ksize also follows the layout of
    # the data. Here we have a pooling window of 2, and a stride of 2.
    pool = tf.nn.max_pool3d(relu,
                          ksize=[1, 2, 2, 2, 1],
                          strides=[1, 2, 2, 2, 1],
                          padding='SAME')
    
    
    conv = tf.nn.conv3d(pool,
                        conv2_weights,
                        strides=[1, 1, 1, 1, 1],
                        padding='SAME')
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
    pool = tf.nn.max_pool3d(relu,
                          ksize=[1, 2, 2, 2, 1],
                          strides=[1, 2, 2, 2, 1],
                          padding='SAME')

    # Reshape the feature map cuboid into a 3D matrix to feed it to the
    # fully connected layers.
    pool_shape = pool.get_shape().as_list()
    reshape = tf.reshape(
        pool,
        [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3] * pool_shape[4]])
  
    # Fully connected layer. Note that the '+' operation automatically
    # broadcasts the biases.
    hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)

    # Add a 50% dropout during training only. Dropout also scales
    # activations such that no rescaling is needed at evaluation time.
    if train:
        hidden = tf.nn.dropout(hidden, 0.5, seed=SEED)
    return tf.matmul(hidden, fc2_weights) + fc2_biases

print('Done')

In [ ]:
# Training computation: logits + cross-entropy loss.
logits = model(train_data_node, True)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
  logits, train_labels_node))

# L2 regularization for the fully connected parameters.
regularizers = (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases) +
                tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases))
# Add the regularization term to the loss.
loss += 5e-4 * regularizers

# Optimizer: set up a variable that's incremented once per batch and
# controls the learning rate decay.
batch = tf.Variable(0)
# Decay once per epoch, using an exponential schedule starting at 0.01.
learning_rate = tf.train.exponential_decay(
  0.01,                # Base learning rate.
  batch * BATCH_SIZE,  # Current index into the dataset.
  train_size,          # Decay step.
  0.95,                # Decay rate.
  staircase=True)
# Use simple momentum for the optimization.
optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=batch)

# Predictions for the minibatch, validation set and test set.
train_prediction = tf.nn.softmax(logits)
# We'll compute them only once in a while by calling their {eval()} method.
validation_prediction = tf.nn.softmax(model(validation_data_node))
test_prediction = tf.nn.softmax(model(test_data_node))

print('Done')

In [ ]:
# Create a new interactive session that we'll use in
# subsequent code cells.
s = tf.InteractiveSession()

# Use our newly created session as the default for 
# subsequent operations.
s.as_default()

# Initialize all the variables we defined above.
tf.initialize_all_variables().run()

In [ ]:
BATCH_SIZE = 60

# Grab the first BATCH_SIZE examples and labels.
batch_data = train_data[:BATCH_SIZE, :, :, :]
batch_labels = train_labels[:BATCH_SIZE]

# This dictionary maps the batch data (as a numpy array) to the
# node in the graph it should be fed to.
feed_dict = {train_data_node: batch_data,
             train_labels_node: batch_labels}

# Run the graph and fetch some of the nodes.
_, l, lr, predictions = s.run(
  [optimizer, loss, learning_rate, train_prediction],
  feed_dict=feed_dict)

print('Done')